In [1]:
from humanfriendly import prompts
import sys
from pathlib import Path
import logging
import os
from rich.console import Console
from rich.markdown import Markdown
from rich.measure import Measurement
from rich.table import Table
import constants
import ArgConfigParse
import collections.abc
import inspect

In [2]:
logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG,
                    format='(%(funcName)-10s):%(lineno)s: %(levelname)-5s %(message)s')
#                     datefmt='%m-%d %H:%M')


In [ ]:
logging.root.setLevel(logging.DEBUG)

In [ ]:
logging.root.setLevel(logging.INFO)

In [ ]:
# class Menu:
#     def __init__(self):
#         self.menu = {}

#     def add_function(self, item_name, function, args=[], kwargs={}):
#         self.menu[item_name] = {'function': function, 'args': args, 'kwargs': kwargs}
    
#     def get_item(self, choice):
#         if isinstance(self.menu[choice], dict) and 'function' in self.menu[choice].keys():
#             val = self._execute(choice)
#             return choice, val
#         if choice not in self.menu.keys():
#             return False, None
#         if choice in self.menu.keys():
#             return choice, self.menu[choice]
    
#     def add_item(self, item_name, return_value=None):
#         if return_value:
#             self.menu[item_name] = return_value
#         else:
#             self.menu[item_name] = item_name
            
#     def add_return(self, item_name='← Return to previous menu'):
#         self.add_item(item_name, return_value='_cancel')
    
#     def add_quit(self, item_name='[X] Quit [X]', message='User exit'):
#         self.add_function(item_name, doExit, [message, 0])
    
#     def _execute(self, choice):
#         func = self.menu[choice]['function']
#         val = func(*self.menu[choice]['args'], **self.menu[choice]['kwargs'])
#         return val
        
        

In [3]:
def dict_merge(dct, merge_dct):
    """ Recursive dict merge. Inspired by :meth:``dict.update()``, instead of
    updating only top-level keys, dict_merge recurses down into dicts nested
    to an arbitrary depth, updating keys. The ``merge_dct`` is merged into
    ``dct``.
    :param dct: dict onto which the merge is executed
    :param merge_dct: dct merged into dct
    :return: None
    """
    for k, v in merge_dct.items():
        if (k in dct and isinstance(dct[k], dict)
                and isinstance(merge_dct[k], collections.abc.Mapping)):
            dict_merge(dct[k], merge_dct[k])
        else:
            dct[k] = merge_dct[k]

In [10]:
class MenuWindow:
    def __init__(self):
        self.items = {}
        self.console = Console()
        self.set_header('''# Title Goes Here''')
    
    def set_header(self, header):
        self.header = Markdown(header)
        
    def show_header(self):
        self.console.print(self.header) 
        
    
    def add_item(self, item_name, return_value=None):
        if return_value:
            self.items[item_name] = return_value
        else:
            self.items[item_name] = item_name
            
    def add_function(self, item_name, function, args=[], kwargs={}):
        self.items[item_name] = {'function': function, 'args': args, 'kwargs': kwargs}
    
    def get_item(self, choice):
        logging.debug(inspect.stack()[1].function)
        
        logging.info(f'getting item: {choice}')
        if isinstance(self.items[choice], dict) and 'function' in self.items[choice].keys():
            logging.debug('is function')
            val = self._execute(choice)
            logging.debug(f'{choice} returned with: {val}')
            return choice, val
        if choice not in self.items.keys():
            logging.warning('item not found!')
            return False, None
        if choice in self.items.keys():
            logging.debug('is item')
            return choice, self.items[choice]
    
    def _execute(self, choice):
        func = self.items[choice]['function']
        val = func(*self.items[choice]['args'], **self.items[choice]['kwargs'])
        return val
    
    def show_menu(self, settings={}):
        logging.debug(inspect.stack()[1].function)

        self.show_header()
        choice = prompts.prompt_for_choice(self.items)
        
        choice, values = self.get_item(choice)
        
        return choice, values
    
    def add_return(self, item_name='← Return to previous menu'):
        self.add_item(item_name, return_value='_cancel')
    
    def add_quit(self, item_name='[X] Quit [X]', message='User exit'):
        self.add_item(item_name, return_value='_quit')

In [11]:
def get_gd(config):
    logging.debug(inspect.stack()[1].function)
    menu = MenuWindow()
    
    try:
        google_drive_entry = Path(config['google_drive_entry']).resolve()
    except KeyError as e:
        google_drive_entry = Path('/Volumes/GoogleDrive/Shared drives/').resolve()
        config['google_drive_entry'] = google_drive_entry
        logging.warning(f'"google_drive_entry" missing from configuration. Set to: {google_drive_entry}')
        
    try:
        google_drive = config['google_drive']
    except KeyError as e:
        logging.info('google_drive not set')
        google_drive = None

    # get a list of dirs
    logging.debug(f'google drive entry point {google_drive_entry}')
    dirs = next(os.walk(google_drive_entry))[1]  
    logging.info(f'found {len(dirs)} shared drives')
    writeable_dirs = {}
    
    # add dirs to menu
    for d in dirs:
        if os.access(google_drive_entry/d, os.W_OK):
            writeable_dirs[d] = google_drive_entry/d
            
    logging.info(f'using {len(writeable_dirs)} shared drives ')
    
    
    for d in sorted(writeable_dirs.keys()):
        menu.add_item(d, writeable_dirs[d])
    
    menu.add_return()
    menu.add_quit()
    
    
    choice, values, = menu.show_menu()
    logging.debug('menu returns: choice, values')
    logging.debug(choice)
    logging.debug(values)
    
    if values == '_quit':
        choice = '_quit'
        values = config
    
    config['google_drive'] = Path(values)
    
    # nest this appropriately for return 
    config = {'settings': config}
    
#     logging.debug('returning: choice, values')
#     logging.debug(choice)
#     logging.debug(values)
    
    logging.debug('returning: choice, config')
    logging.debug(choice)
    logging.debug(config)
    
    return choice, config

In [12]:
# # get_gd({})
# config = {'settings': {'google_drive_entry': '/Volumes/GoogleDrive/Shared drives/'}, 'base_config': {}}

# m = MenuWindow()
# m.add_function('set cf', get_gd, kwargs={'config': config['settings']})
# m.add_function('foobar', foo)
# m.add_item('dummy')

# m.show_menu()

In [13]:
def main_menu(config={}):
    # config = {'settings': {'google_drive_entry': '/Volumes/GoogleDrive/Shared drives/'}, 'base_config': {}}

    menu = MenuWindow()

    menu.add_function('Set Cummulative Folder', get_gd, kwargs={'config': config['settings']})

    menu.add_return()
    menu.add_quit()

    logging.debug(f'{menu.items}')

    choice, values = menu.show_menu()
    
    # if the value returned is a tuple due to a sub menu returning values
    # only preserve the 1th element
    # this is cludgy and should be handled elsewhere
    if isinstance(values, (tuple, list)):
        values = values[1]


    # return only _cancel for the value and choice
    if values == '_cancel':
        choice = values
        values = values

    # return _quit and the current config
    if values == '_quit':
        choice = values
        values = config
    # return the merged configuration in all other cases
    else:
        dict_merge(config, values)
    

    return choice, config
    

In [14]:
c, v = main_menu({'settings': {'google_drive_entry': '/Volumes/GoogleDrive/Shared drives/'}, 'base_config': {}})

(main_menu ):11: DEBUG {'Set Cummulative Folder': {'function': <function get_gd at 0x112662310>, 'args': [], 'kwargs': {'config': {'google_drive_entry': '/Volumes/GoogleDrive/Shared drives/'}}}, '← Return to previous menu': '_cancel', '[X] Quit [X]': '_quit'}
(show_menu ):45: DEBUG main_menu


(prompt_for_choice):190: DEBUG Requesting interactive choice on terminal (options are 'Set Cummulative Folder', '← Return to previous menu' and '[X] Quit [X]') ..



  1. Set Cummulative Folder
  2. ← Return to previous menu
  3. [X] Quit [X]
 
 Enter your choice as a number or unique substring (Control-C aborts): 1



(prompt_for_choice):200: DEBUG Option ('Set Cummulative Folder') selected by numeric reply (1).
(get_item  ):24: DEBUG show_menu
(get_item  ):26: INFO  getting item: Set Cummulative Folder
(get_item  ):28: DEBUG is function
(get_gd    ):2: DEBUG _execute
(get_gd    ):15: INFO  google_drive not set
(get_gd    ):19: DEBUG google drive entry point /Volumes/GoogleDrive/Shared drives
(get_gd    ):21: INFO  found 19 shared drives
(get_gd    ):29: INFO  using 10 shared drives 
(show_menu ):45: DEBUG get_gd


(prompt_for_choice):190: DEBUG Requesting interactive choice on terminal (options are 'ASH Med Care Plans ', 'ASH PowerSchool Learning Resources', 'ASH Student Cumulative Folders', 'ASH Summer 2020 Think Tanks', 'ASH Transportation', 'Data Processing Systems at ASH', 'IT Blabla I', 'IT Support Internal', 'ITM-ITCC Internal', 'PS12 IT Vertical Shared Drive', '← Return to previous menu' and '[X] Quit [X]') ..



  1. ASH Med Care Plans 
  2. ASH PowerSchool Learning Resources
  3. ASH Student Cumulative Folders
  4. ASH Summer 2020 Think Tanks
  5. ASH Transportation
  6. Data Processing Systems at ASH
  7. IT Blabla I
  8. IT Support Internal
  9. ITM-ITCC Internal
  10. PS12 IT Vertical Shared Drive
  11. ← Return to previous menu
  12. [X] Quit [X]
 
 Enter your choice as a number or unique substring (Control-C aborts): 7



(prompt_for_choice):200: DEBUG Option ('IT Blabla I') selected by numeric reply (7).
(get_item  ):24: DEBUG show_menu
(get_item  ):26: INFO  getting item: IT Blabla I
(get_item  ):36: DEBUG is item
(get_gd    ):40: DEBUG menu returns: choice, values
(get_gd    ):41: DEBUG IT Blabla I
(get_gd    ):42: DEBUG /Volumes/GoogleDrive/Shared drives/IT Blabla I
(get_gd    ):57: DEBUG returning: choice, config
(get_gd    ):58: DEBUG IT Blabla I
(get_gd    ):59: DEBUG {'settings': {'google_drive_entry': '/Volumes/GoogleDrive/Shared drives/', 'google_drive': PosixPath('/Volumes/GoogleDrive/Shared drives/IT Blabla I')}}
(get_item  ):30: DEBUG Set Cummulative Folder returned with: ('IT Blabla I', {'settings': {'google_drive_entry': '/Volumes/GoogleDrive/Shared drives/', 'google_drive': PosixPath('/Volumes/GoogleDrive/Shared drives/IT Blabla I')}})


In [ ]:
# def xmain_menu(config={}):
#     menu = Menu()
       
#     menu.add_function('Set Cummulitive Folder', get_google_drive, kwargs={'config': config['settings']})
#     menu.add_function('Choose Student Export', choose_student_export, kwargs={'config': config})
#     menu.add_item('Quit', '_quit' )
    
#     choice = prompts.prompt_for_choice(menu.menu)
    
#     # get the name or the return value from function
#     subChoice, values = menu.get_item(choice) 
    
#     if values == '_quit':
#         return '_quit', config
    
#     if values:
#         newConfig = ArgConfigParse.merge_dict(config, values)
#     else:
#         newConfig = config
    

    
#     return subChoice, newConfig
        

In [ ]:
def doExit(message=None, exit=0):
    logging.debug('doExit')
    if not message:
        message = ''
    if exit > 0:
        logging.warning(message)
        print(f'\nExit: {message}')
        sys.exit(exit)
    else:
        print(f'\n{message}')
        sys.exit(exit)

In [ ]:
# def choose_new_dir():
    
#     dirs = ['Desktop', 'Documents', 'Downloads']
#     menu = Menu()
#     MARKDOWN = f'''
# # Choose a New Directory
# Choose one of the following directories to search for Student Exports
# '''
#     md = Markdown(MARKDOWN)
#     console = Console()
#     console.print(md)           

#     b = Path('~/').expanduser()
    
#     for d in dirs:
#         d = b/Path(d)
#         print(d.name, d)
#         menu.add_item(str(d.name), d)
    
#     menu.add_return()
#     menu.add_quit()
    
#     choice = prompts.prompt_for_choice(menu.menu)
#     return menu.get_item(choice)
        

In [ ]:
# def choose_student_export(config={}):
#     last_folder = Path('~/').expanduser()
#     try:
#         last_folder = Path(config[last_folder])
#     except KeyError:
#         last_folder = Path('~/Downloads').expanduser()
    
#     file_glob = sorted(last_folder.glob('*'))
    
#     menu = Menu()
#     for f in file_glob:
#         if f.is_file():
#             menu.add_item(f.name, f)
#     menu.add_return()
#     menu.add_quit()
    
#     choice = prompts.prompt_for_choice(menu.menu)
    
#     print('#'*40)
#     print(config)
#     print('#'*40)
    
#     config['__runtime']['student_export'] = menu.get_item(choice)[1]
#     return config
# #     return menu.get_item(choice['__runtime']={student_folder})
        
    
    

In [ ]:
def main():    
    appName = constants.appName
    develName = constants.develName
    userPath = constants.userPath
    version = constants.version
    configFile = constants.configFile

    baseConfig = Path(configFile).resolve()
    userConfig = Path('~/.config/').expanduser()/userPath/configFile


    configParser = ArgConfigParse.ConfigFile([baseConfig, userConfig])
    config = configParser.parse_config()
    config['__runtime'] = {}
    config['__runtime']['student_export'] = None
    

    logging.root.setLevel(config['base_config']['log_level'])
    
    
    MARKDOWN=f'''
# portfolioCreator Version: {version}
    
Create student Cumulative Folders in Google Drive using data from PowerSchool Learning
    
    '''
    
    # display the header text
    md = Markdown(MARKDOWN)
    
    
    while True:
        console = Console()
        console.print(md)    


        # display the values
        table = Table(title='Current Settings')
        table.add_column('Setting', style='blue', no_wrap=True)
        table.add_column('Current Setting', style='magenta')
        table.add_row(f'Cumulative Folder Location', f'{config["settings"]["google_drive"]}' )
        table.add_row(f'Debug Level', f'{config["base_config"]["log_level"]}' )                  
        table.add_row(f'Student Export File', f'{config["__runtime"]["student_export"]}')
        console.print(table)    
    
        choice, values = main_menu(config)
        if choice == '_cancel':
            logging.debug('_cancel')
        
        if choice == '_quit':
            ArgConfigParse.write(config, userConfig)
            break
    
        print(config)
                      
    
    print('add cleanup here')    
    return config

In [ ]:
f = main()


In [ ]:
f